In [1]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
print("Working directory")
%cd $module_path

Working directory
/home/efrem/dev/heatflask/backend/heatflask


In [21]:
# %%writefile Strava.py
"""
***  For Jupyter notebook ***

Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:

  * %%cython --annotate
      Compile and run the cell

  * %load Strava.py
     Load Strava.py file into this (empty) cell

  * %%writefile Strava.py
      Write the contents of this cell to Strava.py

"""

import os
import time
import aiohttp
from logging import getLogger
import urllib
import msgpack
import polyline
import asyncio
import datetime

import StreamCodecs

log = getLogger(__name__)
log.propagate = True


STRAVA_DOMAIN = "https://www.strava.com"
STALE_TOKEN = 300

# Client class takes care of refreshing access tokens
class AsyncClient:
    def __init__(self, name, **kwargs):
        self.name = name
        self.set_state(**kwargs)

    def set_state(
        self, access_token=None, expires_at=None, refresh_token=None, scope=None, **extra
    ):
        self.access_token = access_token
        self.expires_at = int(expires_at)
        self.refresh_token = refresh_token
        self.scope = scope
        self.session = self.new_session()

    def __repr__(self):
        expires_at_str = datetime.datetime.fromtimestamp(self.expires_at)
        return f"<AsyncClient '{self.name}' expires-{expires_at_str}>"

    @property
    def expires_in(self):
        return self.expires_at - round(time.time())

    @property
    def headers(self):
        if self.access_token:
            return {"Authorization": f"Bearer {self.access_token}"}

    def new_session(self):
        return aiohttp.ClientSession(
            STRAVA_DOMAIN, headers=self.headers, raise_for_status=True
        )

    async def update_access_token(self, stale_ttl=STALE_TOKEN, code=None):
        if not (self.refresh_token or code):
            return

        if (code is None) and (self.expires_in > stale_ttl):
            log.debug("%s is valid for %ss", self, self.expires_in)
            return

        log.debug("%s updating access token", self)
        t0 = time.perf_counter()
        new_auth_info = await self.run_func(
            get_access_token, code=code, refresh_token=self.refresh_token
        )

        if not (new_auth_info and new_auth_info.get("refresh_token")):
            return

        await self.session.close()
        self.set_state(**new_auth_info)

        elapsed = time.perf_counter() - t0
        log.info("%s token refresh took %.2f", self.name, elapsed)

        return new_auth_info

    async def run_func(self, func, *args, **kwargs):
        try:
            return await func(self.session, *args, **kwargs)
        except Exception:
            log.exception("%s, %s", self, func)

    def get_athlete(self):
        return self.run_func(get_athlete)

    def get_streams(self, activity_id):
        return self.run_func(get_streams, activity_id)

    def get_activity(self, activity_id):
        return self.run_func(get_activity, activity_id)
    
    def get_index(self):
        return get_index(self.session)

    def get_many_streams(self, activity_ids):
        return get_many_streams(self.session, activity_ids)

# *********************************************************************************************
API_SPEC = "/api/v3"

#
# Authentication
#

AUTH_ENDPOINT = "/oauth/authorize"
AUTH_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "response_type": "code",
    "approval_prompt": "auto",  # or "force"
    "scope": "read,activity:read,activity:read_all",
    "redirect_uri": None,
    "state": None,
}

TOKEN_EXCHANGE_ENDPOINT = "/oauth/token"
TOKEN_EXCHANGE_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "client_secret": os.environ["STRAVA_CLIENT_SECRET"],
    #     "code": None,
    #     "grant_type": "authorization_code",
}


def auth_url(redirect_uri=None, state=None):
    params = {**AUTH_PARAMS, "redirect_uri": redirect_uri, "state": state}
    return STRAVA_DOMAIN + AUTH_ENDPOINT + "?" + urllib.parse.urlencode(params)


# We can get the access_token for a user either with
# a code obtained via authentication, or with a refresh token
async def get_access_token(session=None, code=None, refresh_token=None):
    params = {**TOKEN_EXCHANGE_PARAMS}
    params.update(
        {"grant_type": "authorization_code", "code": code}
        if code
        else {"grant_type": "refresh_token", "refresh_token": refresh_token}
    )
    async with session.post(TOKEN_EXCHANGE_ENDPOINT, params=params) as response:
        rjson = await response.json()
    return rjson


#
# Athlete
#
ATHLETE_ENDPOINT = f"{API_SPEC}/athlete"


async def get_athlete(session):
    log.info("test")
    async with session.get(ATHLETE_ENDPOINT) as response:
        return await response.json()


#
# Streams
#
MAX_STREAMS_ERRORS = 10
POLYLINE_PRECISION = 6
ACTIVITY_STREAM_PARAMS = {
    "keys": "latlng,altitude,time",
    "key_by_type": "true",
    "series_type": "time",
    "resolution": "high",
}


def streams_endpoint(activity_id):
    return f"{API_SPEC}/activities/{activity_id}/streams"


async def get_streams(session, activity_id):
    t0 = time.perf_counter()
    async with session.get(
        streams_endpoint(activity_id), params=ACTIVITY_STREAM_PARAMS
    ) as response:
        rjson = await response.json()

        if not (rjson and ("time" in rjson)):
            log.info("problem with activity %d: %s", activity_id, (response.status, rjson))
            return activity_id, None

        result = msgpack.packb(
            {
                "t": StreamCodecs.rlld_encode(rjson["time"]["data"]),
                "a": StreamCodecs.rlld_encode(rjson["altitude"]["data"]),
                "p": polyline.encode(rjson["latlng"]["data"], POLYLINE_PRECISION),
            }
        )
    elapsed = (time.perf_counter() - t0) * 1000
    log.info("fetching streams for %d took %d", activity_id, elapsed)

    return activity_id, result


def unpack_streams(packed_streams):
    streams = msgpack.unpackb(packed_streams)
    return {
        "time": StreamCodecs.rlld_decode(streams["t"], dtype="u2"),
        "altitude": StreamCodecs.rlld_decode(streams["a"], dtype="i2"),
        "latlng": polyline.decode(streams["p"], POLYLINE_PRECISION),
    }


async def get_many_streams(session, activity_ids):
    request_tasks = [
        asyncio.create_task(get_streams(session, aid)) for aid in activity_ids
    ]
    errors = 0
    for task in asyncio.as_completed(request_tasks):
        try:
            activity_id, result = await task
        except Exception as e:
            log.error(e)
            errors += 1
            if errors > MAX_STREAMS_ERRORS:
                abort_signal = True
        else:
            abort_signal = yield activity_id, result

        if abort_signal:
            log.info("get_many_streams aborted")
            yield
            for task in request_tasks:
                task.cancel()
            await asyncio.wait(request_tasks)
            return


#
# Index pages
#
PER_PAGE = 200
REQUEST_DELAY = 1
ACTIVITY_LIST_ENDPOINT = f"{API_SPEC}/athlete/activities"
params = {"per_page": PER_PAGE}


async def get_activity_index_page(session, p):
    t0 = time.perf_counter()

    async with session.get(ACTIVITY_LIST_ENDPOINT, params={**params, "page": p}, raise_for_status=False) as r:
        status = r.status
        result = await r.json()

    elapsed_ms = round((time.perf_counter() - t0) * 1000)
    log.debug("retrieved page %d in %d", p, elapsed_ms)
    return status, p, result


def page_request(session, p):
    return asyncio.create_task(get_activity_index_page(session, p))


EMPTY_LIST = []
async def get_index(user_session):

    done_adding_pages = False
    tasks = set([page_request(user_session, 1)])
    next_page = 2
    while tasks:

        if not done_adding_pages:
            # Add a page requst task
            tasks.add(page_request(user_session, next_page))
            log.debug("requesting page %d", next_page)
            next_page += 1

        # wait for a moment to check for any completed requests
        finished, unfinished = await asyncio.wait(
            tasks, return_when=asyncio.FIRST_COMPLETED, timeout=REQUEST_DELAY
        )

        for task in finished:
            try:
                status, p, result = task.result()
            except Exception as e:
                log.exception("get_index")
                abort_signal = True
            else:
                if status != 200:
                    log.error("Strava error %s: %s", status, result)
                    abort_signal = True
            
                elif len(result):
                    for A in result:
                        abort_signal = yield A
                        if abort_signal:
                            break
                
                elif not done_adding_pages:
                    done_adding_pages = True
                    log.debug("Done requesting pages (status %d)", status)
                
                    
            if abort_signal:
                log.warning("get_index aborted")
                for task in unfinished:
                    task.cancel()
                await asyncio.wait(unfinished)
                yield
                return
                    
                log.debug("processed %d entries from page %d", len(result), p)
            
        tasks = unfinished


def activity_endpoint(activity_id):
    return f"{API_SPEC}/activities/{activity_id}?include_all_efforts=false"


async def get_activity(user_session, activity_id):
    async with user_session.get(activity_endpoint(activity_id)) as r:
        return await r.json()


#
# Updates (Webhook subscription)
#
SUBSCRIPTION_VERIFY_TOKEN = "heatflask_yay!"
SUBSCRIPTION_ENDPOINT = f"{API_SPEC}/push_subscriptions"
SUBSCRIPTION_PARAMS = {
    "client_id": os.environ["STRAVA_CLIENT_ID"],
    "client_secret": os.environ["STRAVA_CLIENT_SECRET"],
}
CREATE_SUBSCRIPTION_PARAMS = {
    **SUBSCRIPTION_PARAMS,
    "verify_token": SUBSCRIPTION_VERIFY_TOKEN,
    "callback_url": None,
}
VIEW_SUBSCRIPTION_PARAMS = SUBSCRIPTION_PARAMS
DELETE_SUBSCRIPTION_PARAMS = {
    **SUBSCRIPTION_PARAMS,
    "id": None,
}


async def create_subscription(admin_session, callback_url):
    params = {**CREATE_SUBSCRIPTION_PARAMS, "callback_url": callback_url}
    async with admin_session.post(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return await response.json()


# After calling create_subscription, you will receive a GET request at your
# supplied callback_url, whose json body is validation_dict.
#
# Your response must have HTTP code 200 and be of application/json content type.
# and be the return value of this function.
async def verify_subscription(validation_dict):
    if validation_dict.get("hub.verify_token") != SUBSCRIPTION_VERIFY_TOKEN:
        return {"hub.challenge": validation_dict["hub.challenge"]}


async def view_subscription(admin_session):
    async with admin_session.get(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return await response.json()


async def delete_subscription(admin_session, subscription_id=None):
    params = {**DELETE_SUBSCRIPTION_PARAMS, "id": subscription_id}
    async with admin_session.delete(SUBSCRIPTION_ENDPOINT, params=params) as response:
        return response.status == 204


Overwriting Strava.py


In [3]:
# Test Create Client

import aiofiles
import json
import logging

logging.basicConfig(level="DEBUG")

credentials_filename = "__pycache__/test-credentials.json"
async with aiofiles.open(credentials_filename, mode='r') as f:
    contents = await f.read()
    access_info = json.loads(contents)

    
async def update_credentials_file(client_name, access_info):
    async with aiofiles.open(credentials_filename, 'w') as json_file:
        await json_file.write(json.dumps(access_info, indent=2))
    log.info("new credentials saved to %s: %s", credentials_filename, access_info)
    
C = AsyncClient("erensi", **access_info)
C, C.expires_in

(<AsyncClient 'erensi' expires-2022-02-10 03:48:35>, 15935)

In [4]:
# Test refresh access_token

new_info = await C.update_access_token()

# Force refresh token
await C.update_access_token(
    #     code="6fe548d4d967252981ce600ad516b5004c2fc00c",
    stale_ttl=C.expires_in + 1
)

DEBUG:__main__:<AsyncClient 'erensi' expires-2022-02-10 03:48:35> is valid for 15935s
DEBUG:__main__:<AsyncClient 'erensi' expires-2022-02-10 03:48:35> updating access token
INFO:__main__:new credentials saved to __pycache__/test-credentials.json: {'token_type': 'Bearer', 'access_token': '914cfb3185a011322755160e1a07300b4cfcc684', 'expires_at': 1644493715, 'expires_in': 15935, 'refresh_token': '9c93b808694a784ea1a3dd4c689ba7a5240ccb46'}
INFO:__main__:erensi token refresh took 0.39


{'token_type': 'Bearer',
 'access_token': '914cfb3185a011322755160e1a07300b4cfcc684',
 'expires_at': 1644493715,
 'expires_in': 15935,
 'refresh_token': '9c93b808694a784ea1a3dd4c689ba7a5240ccb46'}

In [5]:
await C.get_athlete()

INFO:__main__:test


{'id': 15972102,
 'username': 'bfef',
 'resource_state': 2,
 'firstname': '👣',
 'lastname': 'Efrem',
 'bio': '* runs barefoot 👣\n\nhttps://www.heatflask.com',
 'city': 'Oakland',
 'state': 'California',
 'country': 'United States',
 'sex': 'M',
 'premium': True,
 'summit': True,
 'created_at': '2016-06-25T03:48:55Z',
 'updated_at': '2021-09-27T06:08:53Z',
 'badge_type_id': 1,
 'weight': 81.22,
 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg',
 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/large.jpg',
 'friend': None,
 'follower': None}

In [6]:
log.setLevel("DEBUG")

gen = C.get_index()
first10 =  [await gen.__anext__() for i in range(10)]
A = first10[0]
A

DEBUG:__main__:requesting page 2
DEBUG:__main__:requesting page 3
DEBUG:__main__:retrieved page 2 in 1665


{'resource_state': 2,
 'athlete': {'id': 15972102, 'resource_state': 1},
 'name': 'Afternoon Run',
 'distance': 12448.9,
 'moving_time': 4648,
 'elapsed_time': 4768,
 'total_elevation_gain': 372.3,
 'type': 'Run',
 'workout_type': 0,
 'id': 5364354778,
 'start_date': '2021-05-27T00:35:28Z',
 'start_date_local': '2021-05-26T17:35:28Z',
 'timezone': '(GMT-08:00) America/Los_Angeles',
 'utc_offset': -25200.0,
 'location_city': None,
 'location_state': None,
 'location_country': 'United States',
 'achievement_count': 1,
 'kudos_count': 17,
 'comment_count': 0,
 'athlete_count': 1,
 'photo_count': 0,
 'map': {'id': 'a5364354778',
  'summary_polyline': 'gc|eFnrwhVwAAWH_@^a@z@q@x@YRMET_EPm@fBmAbAArAs@lAETs@A_@Y_AwAoA]_BsAaCKi@d@}Cz@kChAkFL{Bh@}AhCoCFg@AqBfB{F~@{@bCaBtByBbAm@Tc@Lo@XwD\\eAn@sA^Yx@Wl@s@CgAe@_DNqAbCwEvBiFlBsCfAcAxFmJXW|@_@xBg@fAAZK\\[pDuGb@gCX_AtEwHrAaDbE}GpAeDn@}BlAsC|@_AdAo@p@s@p@eAVu@ZqCXw@jAmBhAiA|@cB?u@SiAu@uBGyA`AkCz@kExDuGzBkCdBuC~CgB`DqChCwDhC{@|A[TH?^_@t@Mp@Ad@Jv@NV\\RP\

In [7]:
await gen.asend("stop!")

In [13]:
aid, data = await C.get_streams(A["id"])
streams = unpack_streams(data)
aid, streams

INFO:__main__:fetching streams for 5364354778 took 679


(5364354778,
 {'time': array([   0,    1,    2, ..., 4766, 4767, 4768], dtype=uint16),
  'altitude': array([347, 347, 347, ..., 350, 350, 350], dtype=int16),
  'latlng': [(37.832368, -122.186795),
   (37.832398, -122.186795),
   (37.832423, -122.186797),
   (37.832452, -122.1868),
   (37.832477, -122.186803),
   (37.832498, -122.186807),
   (37.832527, -122.186805),
   (37.832552, -122.186802),
   (37.832583, -122.186798),
   (37.832617, -122.186795),
   (37.832652, -122.186795),
   (37.83268, -122.186793),
   (37.832707, -122.18679),
   (37.832737, -122.18679),
   (37.832755, -122.186792),
   (37.832785, -122.186792),
   (37.832803, -122.186788),
   (37.83283, -122.186792),
   (37.83285, -122.186797),
   (37.83287, -122.186807),
   (37.832897, -122.186823),
   (37.832927, -122.18684),
   (37.832952, -122.186863),
   (37.832975, -122.18689),
   (37.832997, -122.18691),
   (37.833018, -122.18693),
   (37.833038, -122.186947),
   (37.833065, -122.186972),
   (37.833085, -122.186995),
   

In [19]:
ids = [a["id"] for a in first10]
gen = C.get_many_streams(ids)
first5 = [await gen.__anext__() for i in range(5)]
# data = {aid: s async for ais, s in gen}

INFO:__main__:fetching streams for 5353369962 took 462
INFO:__main__:fetching streams for 5325571502 took 548
INFO:__main__:fetching streams for 5353497175 took 570
INFO:__main__:fetching streams for 5342144786 took 591
INFO:__main__:fetching streams for 5303499514 took 611
INFO:__main__:fetching streams for 5364354778 took 664
INFO:__main__:fetching streams for 5326881692 took 726
INFO:__main__:fetching streams for 5298326959 took 746
INFO:__main__:fetching streams for 5341632207 took 781
INFO:__main__:fetching streams for 5309074925 took 796


In [20]:
await gen.asend("quit")

INFO:__main__:get_many_streams aborted
